# TLDR

This notebook does something


In [10]:
import numpy as np
import numpyro
import numpyro.distributions as dist
from numpyro.distributions import DirichletMultinomial

from numpyro.infer import MCMC, NUTS

from jax import random


In [ ]:
# variable = numpyro.sample(variable_name, distribution)
# model: produces an observation given parameters from the prior
# kernel = NUTS(model, args)
# mcmc = MCMC(kernel, args)
# mcmc.run(seed, N = ? )
# mcmc.print_summary()
# mcmc.get_samples(group_by_chain = True/False)

# Predictive(model, mcmc.get_samples())

In [19]:

p = numpyro.sample(
    'p', 
    DirichletMultinomial(concentration=np.array([0.1, 0.1, 0.1])), 
    rng_key=random.PRNGKey(1)
)



In [20]:
p

Array([0, 1, 0], dtype=int32)

In [26]:
def infections(N, lamda, beta, y=None):

    r""" 
    Args:
        N: population size
        lamda: prevalence
        beta: probability of infection given contact with positive
    """

    p = numpyro.sample(
        'p', 
        DirichletMultinomial(concentration=np.array([0.1, 0.1, 0.1])), 
        rng_key=random.PRNGKey(1)
    )

    with numpyro.plate('N', N):
        # for each individual, sample their number of contacts k from the Dirichlet distribution
        # then sample a Bernoulli with probability 1-(1-lambda*beta)^k

        num_contacts = numpyro.sample('num_contacts', dist.Multinomial(probs=p))
        p_inf = 1-(1-lamda*beta)^num_contacts
        numpyro.sample('infected', dist.Bernoulli(probs=p_inf), obs = y)

In [27]:
N=10
lamda = 0.1
beta=0.1
y = np.array([1,1,1,1,0,0,0,0,1,1])
# TODO: there should be more to that; we know the infection status as well as reported number of contacts



nuts_kernel = NUTS(infections)
mcmc = MCMC(nuts_kernel, num_warmup=500, num_samples=1000)
rng_key = random.PRNGKey(0)
mcmc.run(rng_key, N=N, lamda = lamda, beta=beta, y=y, extra_fields=('potential_energy',))

TypeError: xor does not accept dtype float32 at position 0. Accepted dtypes at position 0 are subtypes of integer, bool_.